In [1]:
##Load metadata from file

import ast

d = {}
nary = {}
with open("metadata_trial4.txt", 'r') as f:
    for line in f:
        d = (ast.literal_eval(line))
        nary.update({d['id'] : d})

In [2]:
#Filter terms that have children and aspect

filnary = {}
kilnary = {}
nochild = {}

for key, value in nary.items():
    if value.get('children') != None:
        filnary = ({'id' : value['id'], 'children' : value['children']})
        kilnary.update({key : filnary})
    else:
        filnary = ({'id' : value['id'],})
        nochild.update({key : filnary})

In [3]:
#get parents

for key, value in kilnary.items():
    for item in value['children']:
        if nary[item['id']].get('parent') == None:
            nary[item['id']]['parent'] = []
            nary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})
        else:
            nary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})
            
for key, value in nary.items():
    if value.get('parent') == None:
        value['parent'] = []

In [4]:
#empty children class for nochilds

for key, value in nary.items():
    if value.get('children') == None:
        value['children'] = []

In [5]:
#get rid of the two motherfuckers

for k,v in nary.items():
    for item in v['parent']:
        for item2 in v['children']:
            if item == item2:
                del(v['parent'][v['parent'].index(item)])
                del(v['children'][v['children'].index(item)])

In [6]:
#hashmap

hashmap = {}

for key, value in nary.items():
    hashmap[key] = False

In [7]:
#import owlready2

import types
from owlready2 import *
onto = get_ontology("http://test.org/onto.owl")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [8]:
#define core classes

with onto:
    class therapeutic_agent(Thing):
        namespace = onto
    class biological_process_main(Thing):
        namespace = onto
    class molecular_function_main(Thing):
        namespace = onto
    class drug(Thing):
        namespace = onto

In [9]:
def recursive_classmaker_down(term, counter):
    hashmap[term] = True
    for item in nary[term]['children']:
        if item['relation'] == 'is_a':
            with onto:
                nc = types.new_class(item['id'], (getattr(onto, term),))
            if hashmap[item['id']] == False:
                counter+=1
                print(counter)
                recursive_classmaker_down(item['id'], counter)
                
    return counter

In [10]:
#Using classmaker to make FTC classes

counter = 0
for key, value in nary.items():
    if value['parent'] == [] and value['aspect'] == 'biological_process':
        with onto:
            nc = types.new_class(key, (biological_process_main,))
        if hashmap[key] == False:
            counter+=1
            counter = recursive_classmaker_down(key, counter)
   

2
3
4
5
6
7
8
9
10
11
12
13
4
5
6
7
8
8
9
9
10
10
11
6
7
8
8
9
10
11
11
12
13
9
10
11
12
11
12
13
14
15
16
12
13
14
15
16
10
3
4
5
6
7
8
7
8
8
9
9
5
6
7
6
7
8
9
9
10
11
7
8
9
10
11
11
12
10
11
12
11
12
13
14
15
14
13
14
15
16
17
18
16
17
18
15
16
17
14
15
16
17
18
19
20
21
22
18
19
20
21
19
17
18
19
20
21
22
23
24
9
10
11
12
11
12
13
12
13
13
10
11
12
13
14
14
13
14
15
14
15
16
17
18
19
20
21
22
23
24
25
26
27
20
19
20
21
21
20
21
22
22
17
18
19
20
18
19
20
21
22
23
24
20
21
22
23
24
25
16
17
18
18
19
20
21
21
22
22
23
23
24
15
16
17
18
19
20
21
22
19
20
21
22
18
19
20
21
22
23
21
22
23
22
23
24
25
23
24
25
25
26
26
27
28
24
19
17
18
18
19
20
21
19
20
21
22
23
23
24
25
24
25
26
25
26
27
21
22
23
24
24
25
26
22
23
20
21
16
17
18
19
20
18
19
20
21
20
21
11
12
13
13
14
14
15
16
17
17
18
19
20
21
18
19
20
21
22
23
21
19
20
21
22
16
17
18
18
19
19
17
18
15
16
17
18
19
17
18
19
20
21
22
23
16
17
18
19
20
18
19
19
20
21
22
23
24
23
24
12
13
14
14
15
16
17
18
17
18
18
19
20
21
22
23
23
21
22
2

22
23
24
24
25
26
17
18
19
20
21
22
23
24
25
26
25
24
25
26
26
21
22
23
20
21
22
23
24
25
26
26
24
25
26
27
28
29
25
26
27
28
29
30
31
26
27
28
23
24
25
26
27
28
29
30
29
26
27
28
29
30
31
27
28
29
30
31
32
29
30
31
32
33
34
35
36
37
38
39
30
31
32
33
34
35
36
37
25
26
22
23
24
25
26
27
28
29
28
29
23
24
25
26
27
26
27
28
29
30
30
27
28
29
30
31
32
33
34
35
36
18
19
19
20
20
21
22
23
24
24
25
26
27
28
29
25
26
27
28
23
24
25
25
26
27
26
27
24
25
25
26
26
27
28
27
28
29
29
30
30
31
31
32
33
34
32
33
34
35
28
29
30
31
29
30
22
23
23
24
25
25
26
24
25
26
26
27
27
28
28
25
26
27
28
29
29
30
31
32
33
28
29
30
31
31
32
33
34
34
35
36
36
37
38
29
30
31
32
33
34
35
36
34
35
36
37
38
39
40
41
42
43
44
45
46
40
38
39
40
41
42
39
40
41
27
28
26
27
28
29
30
31
31
32
32
33
34
33
30
31
32
33
34
35
36
37
38
39
40
41
42
43
33
34
35
36
37
38
39
39
40
41
42
43
44
45
46
45
46
35
36
37
38
39
40
37
38
39
40
41
38
21
22
22
23
23
24
25
26
26
27
28
28
27
28
29
30
29
30
30
31
32
32
33
33
31
28
29
30
31
31
32
3

33
34
35
36
33
26
27
28
27
28
29
30
31
32
33
24
25
26
25
26
27
28
28
29
30
30
27
28
29
30
22
23
23
24
25
24
25
26
27
28
29
25
26
27
28
27
28
29
30
31
30
28
29
30
31
32
26
27
28
27
28
29
30
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
31
32
33
34
35
36
29
30
31
32
33
34
28
29
30
31
30
31
31
32
33
34
21
22
23
23
24
25
25
26
27
27
28
28
29
30
31
32
29
30
31
26
27
27
28
29
30
28
29
22
23
24
25
26
23
24
25
26
27
28
27
28
29
30
26
27
28
27
28
28
29
30
31
30
31
32
33
29
30
31
32
33
34
35
36
31
32
33
34
34
32
33
34
35
36
25
26
27
28
28
29
30
26
27
28
29
30
31
32
33
31
29
30
31
30
31
32
33
32
31
32
33
34
33
34
35
34
35
36
37
35
36
37
38
36
37
37
38
38
39
39
27
28
29
30
31
30
31
32
32
33
34
35
33
34
35
34
35
36
37
38
37
38
39
38
39
40
41
42
43
44
45
46
47
48
48
49
50
51
52
49
50
51
52
53
53
54
55
56
57
56
57
58
50
43
44
45
46
47
48
49
50
51
52
51
52
53
54
55
56
55
56
57
57
58
59
58
59
60
61
62
63
64
65
66
66
67
68
69
63
64
65
66
67
67
68
69
46
47
48
49
50
51
50
51
49
50
51
52
53
5

41
23
21
22
23
24
25
26
27
26
27
28
23
24
25
25
24
25
26
27
28
29
26
27
28
29
29
30
31
32
33
34
33
34
35
34
35
31
32
33
34
35
36
27
28
29
30
31
32
33
34
34
35
36
30
31
32
33
34
35
36
34
29
30
31
32
31
32
33
34
25
26
27
26
27
28
22
23
24
25
26
27
25
26
27
26
27
28
29
30
24
25
25
26
23
24
25
26
27
26
27
28
27
28
29
30
31
32
33
33
34
35
31
32
32
33
34
35
35
33
34
35
36
37
37
38
35
36
34
30
29
30
31
32
31
32
33
34
35
36
35
36
37
38
39
40
41
42
34
35
36
37
38
39
40
41
42
43
43
44
44
45
46
47
48
36
37
38
39
40
40
41
41
42
39
40
41
42
43
42
40
41
42
43
44
45
46
47
48
43
44
45
46
47
47
48
49
41
42
43
44
45
46
47
48
49
50
49
50
50
51
52
53
54
53
54
55
54
55
52
53
53
54
55
56
57
56
57
57
58
58
55
56
56
54
55
55
56
57
58
58
59
60
59
60
61
61
62
62
63
64
65
66
67
63
64
65
65
66
67
68
69
70
71
67
68
69
70
71
72
73
74
75
76
64
65
66
67
68
67
68
66
28
29
30
30
31
32
32
29
30
30
31
32
33
34
35
36
36
19
20
21
22
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
40
41
42
43
44
45
46
47
48
49
5

55
55
56
57
58
59
60
61
61
62
63
52
53
54
55
56
57
58
59
60
59
60
54
55
56
57
58
59
60
61
7
8
9
10
11
12
13
14
15
16
17
18
17
10
11
12
12
8
9
10
11
11
12
13
14
15
16
16
17
17
15
16
17
16
17
14
15
16
16
17
17
15
16
17
13
14
15
16
17
17
18
18
16
17
17
18
18
15
16
17
17
16
17
18
18
17
18
18
14
15
16
17
17
18
18
16
17
17
18
18
15
16
17
18
19
18
19
20
17
18
19
20
21
22
23
16
17
12
13
14
13
14
15
16
17
18
17
18
19
20
21
22
23
24
25
26
27
25
26
21
22
23
24
25
26
27
28
26
27
22
23
16
17
18
17
18
19
20
21
22
22
23
24
24
15
9
10
11
12
13
14
15
16
17
18
13
11
12
12
10
11
12
13
14
15
16
14
15
16
17
15
16
17
18
19
20
21
22
21
22
22
16
17
18
19
20
21
22
23
24
25
25
26
26
22
23
24
25
26
27
28
29
30
30
31
32
28
24
19
20
20
21
22
23
21
17
18
19
18
19
20
21
22
23
24
25
26
20
21
22
23
24
25
26
26
27
28
28
29
25
26
27
28
29
29
28
29
30
30
31
32
33
33
34
34
35
32
33
34
33
34
35
34
35
35
36
37
37
38
39
38
39
40
41
40
41
42
41
42
43
39
40
41
42
41
42
43
42
43
44
45
46
43
44
45
46
40
41
42
43
44
44
45
22
23
1

...

40
41
42
43
44
43
42
43
44
45
43
44
45
44
41
42
43
44
45
46
43
44
44
45
46
46
47
48
49
49
50
51
52
53
53
48
49
50
51
52
53
54
53
54
55
56
57
58
59
60
58
59
60
56
57
58
59
60
61
62
63
64
62
63
64
65
63
60
61
59
60
61
61
62
62
63
64
65
65
66
67
68
69
70
71
72
73
71
72
60
61
62
62
63
64
65
66
66
67
63
64
65
65
66
66
67
67
68
69
68
69
69
61
62
62
63
64
65
65
63
64
64
65
66
67
68
69
55
56
57
58
59
60
61
62
63
64
65
66
66
67
68
69
68
65
66
67
68
69
70
71
72
71
66
67
68
61
62
62
63
64
65
66
67
68
69
69
70
68
69
67
68
69
70
71
69
70
70
71
72
73
74
73
74
66
60
61
61
62
63
64
65
66
66
67
68
65
64
65
42
43
44
43
44
45
45
46
44
45
45
46
46
47
48
49
50
51
52
53
54
55
53
54
55
56
57
55
52
53
54
55
53
54
55
51
52
53
53
54
55
56
57
58
56
54
55
55
56
57
58
58
59
60
61
52
53
54
54
55
56
57
58
58
59
55
56
57
57
58
58
59
59
60
60
61
62
61
53
54
55
56
56
55
56
57
57
58
59
60
61
62
63
64
50
51
52
53
54
55
48
49
50
51
52
53
54
55
56
57
58
57
58
52
50
51
52
53
54
55
53
54
55
56
57
55
52
53
54
55
53
54
55
56
5

50
51
51
52
53
54
53
54
55
56
57
41
39
35
36
37
38
37
36
37
38
39
40
40
41
38
39
40
40
41
42
43
44
43
44
42
43
44
43
44
39
40
41
41
42
42
43
40
41
42
43
44
43
44
45
46
42
43
44
45
46
47
48
46
47
48
44
45
43
41
42
42
43
44
45
46
46
47
45
46
47
48
49
48
49
46
47
48
49
48
49
44
45
46
47
47
48
49
46
47
48
49
47
48
49
50
51
50
48
45
46
47
48
48
47
48
49
49
43
44
45
46
44
45
37
38
39
40
41
42
41
42
43
43
44
45
44
45
46
47
48
49
50
42
43
43
44
44
45
45
46
47
47
46
47
48
48
49
47
48
48
49
50
51
40
41
42
42
43
43
44
41
42
42
43
43
44
44
45
39
40
41
42
42
43
44
45
46
47
48
46
47
48
49
50
40
41
42
42
43
44
45
46
44
45
46
47
48
49
50
49
41
42
43
44
45
46
46
47
48
49
44
45
46
47
48
49
50
42
43
44
38
39
40
41
42
41
42
43
42
43
44
45
45
46
47
43
44
44
45
45
46
47
48
49
50
46
47
48
49
40
41
42
43
42
43
43
44
44
41
42
43
44
43
44
45
44
45
46
47
46
47
45
46
47
48
49
48
49
49
50
51
52
53
53
54
55
54
55
55
56
57
58
59
56
57
46
47
48
48
49
50
51
49
50
51
47
48
49
50
49
50
50
51
51
52
53
54
55
56
57
58
59
6

59
60
61
62
55
56
57
58
59
60
61
62
63
32
33
34
35
36
36
37
34
35
35
36
37
38
39
40
41
37
38
39
40
41
42
43
39
40
18
19
20
21
21
22
23
23
20
21
22
23
24
25
26
27
28
28
25
26
27
23
24
24
25
26
22
23
24
25
26
27
28
29
30
31
29
30
31
32
33
30
31
32
33
34
35
36
32
33
34
35
35
36
36
37
33
34
34
35
36
37
38
39
39
40
41
38
39
37
38
39
39
40
38
39
40
41
40
39
40
41
42
40
41
42
43
36
37
38
39
40
40
41
42
43
44
45
46
45
46
47
48
44
45
41
42
43
43
44
44
45
46
42
43
43
44
39
40
41
42
43
44
45
41
42
38
39
40
40
41
42
39
40
41
42
43
44
45
46
44
42
43
43
44
45
44
45
46
47
48
49
47
46
41
42
43
44
42
43
44
43
44
45
46
46
47
48
49
50
48
49
50
44
27
28
29
30
31
30
31
32
33
29
30
31
32
32
33
34
35
36
37
36
37
38
39
40
33
34
35
36
37
38
38
39
39
40
40
41
36
37
37
38
39
40
41
42
42
43
41
40
41
41
42
42
43
44
44
45
46
47
48
49
50
50
51
39
40
41
42
42
43
44
44
45
46
47
48
49
50
51
47
43
44
45
46
47
25
26
27
28
29
30
27
28
29
24
25
26
27
28
29
30
31
32
33
34
35
27
28
29
30
29
28
29
29
30
31
32
33
30
31
31
32
3

163
164
162
163
164
165
163
164
149
150
151
152
151
152
153
154
155
156
157
153
154
155
154
155
156
157
128
129
129
130
131
132
133
134
135
135
136
137
138
134
135
136
137
138
139
140
141
142
143
136
137
138
139
140
141
142
141
142
143
144
145
145
146
147
148
149
148
144
145
146
147
148
149
150
151
152
153
154
153
154
155
156
155
156
157
156
157
158
159
157
158
159
160
161
162
163
163
164
165
166
167
168
161
162
163
164
165
166
167
168
169
170
171
172
173
170
171
172
171
172
162
163
164
165
166
167
167
163
164
165
166
167
167
165
166
167
168
169
170
171
172
173
174
174
175
173
170
171
172
173
174
173
174
175
176
177
178
179
180
180
175
176
177
178
179
180
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
176
177
178
179
180
181
181
182
183
152
153
154
155
156
157
158
159
160
161
162
163
164
163
164
165
166
167
168
169
170
171
172
172
169
170
171
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
186
185
186
187
188
189
190
191
191
192
193
193
194


KeyboardInterrupt: 

In [11]:
len((list(onto.classes())))

29824

In [34]:
(list(onto.classes()))

[onto.therapeutic_agent,
 onto.biological_process_main,
 onto.molecular_function_main,
 onto.drug,
 onto.GO:0003674,
 onto.GO:0045182,
 onto.GO:0090079,
 onto.GO:0000900,
 onto.GO:0008494,
 onto.GO:0008135,
 onto.GO:0003743,
 onto.GO:0008079,
 onto.GO:0003747,
 onto.GO:0016150,
 onto.GO:0016149,
 onto.GO:0003746,
 onto.GO:0004694,
 onto.GO:0030371,
 onto.GO:0000901,
 onto.GO:0045183,
 onto.GO:0140110,
 onto.GO:0001072,
 onto.GO:0003711,
 onto.GO:0003700,
 onto.GO:0001217,
 onto.GO:0001227,
 onto.GO:0001216,
 onto.GO:0001228,
 onto.GO:0098531,
 onto.GO:0004879,
 onto.GO:0000981,
 onto.GO:0034246,
 onto.GO:0043856,
 onto.GO:0001073,
 onto.GO:0140223,
 onto.GO:0001181,
 onto.GO:0000995,
 onto.GO:0016251,
 onto.GO:0001070,
 onto.GO:0001134,
 onto.GO:0001186,
 onto.GO:0001004,
 onto.GO:0001135,
 onto.GO:0016987,
 onto.GO:0003712,
 onto.GO:0003713,
 onto.GO:0030374,
 onto.GO:0030375,
 onto.GO:0003714,
 onto.GO:0016989,
 onto.GO:0140313,
 onto.GO:0140315,
 onto.GO:0140319,
 onto.GO:0140320,
 

In [40]:
len(list(onto.classes()))

42976

In [41]:
for i in l:
    if str(i)[5:7] == "GO":
        for item in nary[str(i)[5:]]['children']:
            for ancestor in list(i.ancestors()):
                if item['id'] == str(ancestor)[5:]:
                    print(item['id'])

GO:0009603


In [9]:
#conflict checker

for k,v in nary.items():
    for item in v['parent']:
        for item2 in v['children']:
            if item == item2:
                print(item)

In [10]:
#conflict checker

for k,v in nary.items():
    for item in v['parent']:
        for item2 in v['children']:
            if item == item2:
                del(v['parent'][v['parent'].index(item)])
                del(v['children'][v['children'].index(item)])

In [25]:
l = list(onto.classes())

In [40]:
l[5].ancestors()

{onto.GO:0003674,
 onto.GO:0005554,
 onto.GO:0045182,
 onto.molecular_function_main,
 owl.Thing}

In [22]:
list(onto.biological_process.subclasses())

[onto.biological_cess, onto.molecular_func, onto.something_binding]

In [16]:
abc = onto.behavior("test_abc")

In [9]:
onto.save(file = "t2test3", format = "rdfxml")